In [ ]:
!pip install konlpy
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path as osp
from konlpy.tag import Okt
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/DL_project/Sub_Word/preprocessed.csv')
train

,Unnamed: 0,emotion,Q,emotion_label,preproc,prep
0,0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...,1,일은 왜 해도 해도 끝이 없을까 화가 난다 그냥 내가 해결하는 게 나아 남들한테 부...,"['끝', '화가', '나다', '그냥', '해결', '게', '나아', '남', ..."
1,1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,1,이번 달에 또 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 최...,"['달', '급여', '깎다', '물가', '월급', '자꾸', '깎다', '너무'..."
2,2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,1,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...,"['회사', '신입', '들어오다', '말투', '거슬리다', '그렇다', '애',..."
3,3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,1,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은 데 정말 분하고 섭...,"['직장', '막내', '이유', '온갖', '심부름', '일도', '많다', '정..."
4,4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...,1,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사인 나에게 먼...,"['입사', '신입사원', '나르다', '무시', '너무', '화가', '상사', ..."
...,...,...,...,...,...,...
56999,56999,슬픔,병원에서 친하게 지내던 환자가 암으로 죽어서 너무 슬퍼. 신부전으로 입원했을 때 많...,4,병원에서 친하게 지내던 환자가 암으로 죽어서 너무 슬퍼 신부전으로 입원했을 때 많은...,"['병원', '친하다', '지내다', '환자', '암', '죽다', '너무', '슬..."
57000,57000,상처,천식 때문에 가족들 두고 산에서 혼자 사는데 외롭네. 가족들이랑 같이 지내고 싶은데...,3,천식 때문에 가족들 두고 산에서 혼자 사는데 외롭네 가족들이랑 같이 지내고 싶은데 ...,"['천식', '가족', '두다', '산', '살다', '외롭다', '가족', '지내..."
57001,57001,분노,오늘 식당에서 서빙을 하는데 젊은 청년들이 반말하길래 기분이 나빠 나도 반말을 하며...,1,오늘 식당에서 서빙을 하는데 젊은 청년들이 반말하길래 기분이 나빠 나도 반말을 하며...,"['오늘', '식당', '서빙', '젊다', '청년', '반말', '기분', '나쁘..."
57002,57002,상처,최 영감이 노인들에게 나를 이상한 사람으로 이야기해서 답답하고 억울해. 최 영감은 ...,3,최 영감이 노인들에게 나를 이상한 사람으로 이야기해서 답답하고 억울해 최 영감은 나...,"['최', '영감', '노인', '나르다', '이상하다', '이야기', '답답하다'..."


In [ ]:
# 불용어 처리(리스트 받아서, 불용어 제거된 리스트 반환)
#def stop_words_remover(list):
#    return [word for word in list if not word in stopwords]

#train['preproc'] = train['preproc'].apply(stop_words_remover)
#en(train['preproc'].sum())

In [ ]:
f = open('/content/drive/MyDrive/DL_project/Sub_Word/123.txt','a')
for i in range(len(train)):
  f.write('{}\n\n'.format(train['preproc'][i]))
f.close()

In [ ]:
import sentencepiece as spm

input_file = '/content/drive/MyDrive/DL_project/Sub_Word/123.txt'
vocab_size = 32000
model_name = 'subword_tokenizer_kor'
model_type = 'bpe'
user_defined_symbols = '[PAD],[UNK],[CLS],[SEP],[MASK],[UNK1],[UNK2],[UNK3],[UNK4],[UNK5]'

input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --user_defined_symbols=%s --model_type=%s'
cmd = input_argument%(input_file, model_name, vocab_size,user_defined_symbols, model_type)

spm.SentencePieceTrainer.Train(cmd)

In [ ]:
sp = spm.SentencePieceProcessor()
sp.Load("/content/drive/MyDrive/DL_project/Sub_Word/subword_tokenizer_kor.model")

True

In [ ]:
ser = pd.Series()
for i in range(len(train)):
  ser.loc[i] = sp.EncodeAsPieces(train['preproc'][i])
ser

<ipython-input-16-973c710efbb8>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ser = pd.Series()


0        [▁일은, ▁왜, ▁해도, ▁해도, ▁끝이, ▁없을까, ▁화가, ▁난다, ▁그냥, ...
1        [▁이번, ▁달에, ▁또, ▁급여가, ▁깎였어, ▁물가는, ▁오르는데, ▁월급, 만...
2        [▁회사에, ▁신입이, ▁들어왔는데, ▁말투가, ▁거슬려, ▁그런, ▁애를, ▁매일...
3        [▁직장에서, ▁막내, 라는, ▁이유로, ▁나에게만, ▁온갖, ▁심부름을, ▁시켜,...
4        [▁얼마, ▁전, ▁입사한, ▁신입사원이, ▁나를, ▁무시하는, ▁것, ▁같아서, ...
                               ...                        
56999    [▁병원에서, ▁친하게, ▁지내던, ▁환자가, ▁암으로, ▁죽어서, ▁너무, ▁슬퍼...
57000    [▁천식, ▁때문에, ▁가족들, ▁두고, ▁산, 에서, ▁혼자, ▁사는데, ▁외롭네...
57001    [▁오늘, ▁식당에서, ▁서, 빙, 을, ▁하는데, ▁젊은, ▁청년들이, ▁반말, ...
57002    [▁최, ▁영감이, ▁노인들에게, ▁나를, ▁이상한, ▁사람으로, ▁이야기해서, ▁...
57003    [▁이렇게, ▁좋은, ▁운동, ▁시설에서, ▁경로, ▁우, 대로, ▁운동할, ▁수,...
Length: 57004, dtype: object

In [ ]:
train['subword']= ser
train['subword']

0        [▁일은, ▁왜, ▁해도, ▁해도, ▁끝이, ▁없을까, ▁화가, ▁난다, ▁그냥, ...
1        [▁이번, ▁달에, ▁또, ▁급여가, ▁깎였어, ▁물가는, ▁오르는데, ▁월급, 만...
2        [▁회사에, ▁신입이, ▁들어왔는데, ▁말투가, ▁거슬려, ▁그런, ▁애를, ▁매일...
3        [▁직장에서, ▁막내, 라는, ▁이유로, ▁나에게만, ▁온갖, ▁심부름을, ▁시켜,...
4        [▁얼마, ▁전, ▁입사한, ▁신입사원이, ▁나를, ▁무시하는, ▁것, ▁같아서, ...
                               ...                        
56999    [▁병원에서, ▁친하게, ▁지내던, ▁환자가, ▁암으로, ▁죽어서, ▁너무, ▁슬퍼...
57000    [▁천식, ▁때문에, ▁가족들, ▁두고, ▁산, 에서, ▁혼자, ▁사는데, ▁외롭네...
57001    [▁오늘, ▁식당에서, ▁서, 빙, 을, ▁하는데, ▁젊은, ▁청년들이, ▁반말, ...
57002    [▁최, ▁영감이, ▁노인들에게, ▁나를, ▁이상한, ▁사람으로, ▁이야기해서, ▁...
57003    [▁이렇게, ▁좋은, ▁운동, ▁시설에서, ▁경로, ▁우, 대로, ▁운동할, ▁수,...
Name: subword, Length: 57004, dtype: object